# Automated ML for time series predicion

We used one of the dataset in [Numenta Anomaly Benchmark (NAB)](https://github.com/numenta/NAB) for demo, i.e. NYC taxi passengers dataset, which contains 10320 records, each indicating the total number of taxi passengers in NYC at a corresonponding time spot. 

In [ ]:
%load_ext autoreload
%autoreload 2

## 0. Helper function definations

In [ ]:
# plot the predicted values and actual values (for the test data)
def plot_result(test_df, pred_df):
    # target column of dataframe is "value"
    # past sequence length is 50
    pred_value = pred_df["value"].values
    true_value = test_df["value"].values[50:]
    fig, axs = plt.subplots()

    axs.plot(pred_value,color='red', label='predicted values')
    axs.plot(true_value,color='blue', label='actual values')
    axs.set_title('the predicted values and actual values (for the test data)')

    plt.xlabel('test data index')
    plt.ylabel('number of taxi passengers')
    plt.legend(loc='upper left')
    plt.show()

## 1. load data

We have implemented load_nytaxi_data_df method to load nyc taxi data into train/val/test dataframe. You can change val_split_ratio, test_split_ratio and data_path as you want. Train/val/test dataframe all have two columns: "datetime" and "value", where "value" column is the target to predict.

In [ ]:
import os
import pandas as pd
import numpy as np

import matplotlib
matplotlib.use('Agg')
%pylab inline
import seaborn
import matplotlib.dates as md
from matplotlib import pyplot as plt

In [ ]:
# load nyc taxi data
from zoo.automl.common.util import load_nytaxi_data_df
train_df, val_df, test_df = load_nytaxi_data_df(val_split_ratio=0.1, test_split_ratio=0.1)

In [ ]:
train_df.describe()

In [ ]:
train_df.head(5)

In [ ]:
# shape of the dataframe
print("The shape of train_df is", train_df.shape)
print("The shape of val_df is", val_df.shape)
print("The shape of test_df is", test_df.shape)

In [ ]:
# visualisation of anomaly throughout time in train_df
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

fig, ax = plt.subplots(figsize=(12, 5))
# pd.plotting.deregister_matplotlib_converters()

ax.plot(train_df['datetime'], train_df['value'], color='blue', linewidth=0.6)
ax.set_title('NYC taxi passengers throughout time')

plt.xlabel('datetime')
plt.xticks(rotation=45) 
plt.ylabel('The Number of NYC taxi passengers')
plt.legend(loc='upper left')
plt.show()

## 2. Train and validation

You can use analytices zoo automl to predict time series data by simply define a `TimeSequencePredictor`. Currently, our automl support searching feature combinations in a predefined feature space, and searching model hyper-parameters. Model selection is not supported yet.  

We use feature tools to generate features from the given datetime. The generated features are \['HOUR', 'DAY', 'MONTH'. 'IS_AWAKE', 'IS_BUSY_HOURS'\]. Our feature space comprises these generated features as well as the original inputs such as \['datetime','value'\].  

We use a vanilla LSTM model to predict time series data. The model search space contains the unit num of lstm layers ,dropout rate and hyper-parameters for training, such as batch_size, learning rate, etc. 

Currently, We use RNN to learn from 50 previous values, and predict just the 1 next value. You can specify the sequence length to predict while creating `TimeSequencePredictor` with arg: `future_seq_len`. The previous sequence length will be  considered as a tunable variable in our future version.

In [ ]:
# build time sequence predictor
from zoo.automl.regression.time_sequence_predictor import TimeSequencePredictor

# you need to specify the name of datetime column and target column
# The default names are "datetime" and "value" respectively.
tsp = TimeSequencePredictor(dt_col="datetime",
                            target_col="value",
                            extra_features_col=None)

In [ ]:
%%time
# fit train_df and validate with val_df, return the best trial as pipeline.
# the default trail num is 10, need about 5.5 mins.
pipeline = tsp.fit(train_df,
                   validation_df=val_df,
                   metric="mean_squared_error")
print("Training completed.")

## 3. Test

In [ ]:
# predict test_df with the best trial
pred_df = pipeline.predict(test_df)

In [ ]:
pred_df.head(5)

In [ ]:
# prediction value start from 50
test_df[50:55]

In [ ]:
# evaluate test_df
mse, rs = pipeline.evaluate(test_df, metric=["mean_squared_error", "r_square"])
print("Evaluate: the mean square error is", mse)
print("Evaluate: the r_squared value is", rs)

In [ ]:
# plot the predicted values and actual values
plot_result(test_df, pred_df)

## 4. save and restore

We provide save and restore interface to save the pipeline with the best trial for easily rebuilding.

In [ ]:
# save the pipeline with best trial
saved_pipeline_file = "../../../saved_pipeline/"
pipeline.save(saved_pipeline_file)

In [ ]:
# create a new pipeline and specify the pipeline path to restore
from zoo.automl.pipeline.time_sequence import TimeSequencePipeline

new_pipeline = TimeSequencePipeline()
new_pipeline.restore(saved_pipeline_file)

In [ ]:
# you can do predict and evaluate again
# to compare results before and after restore, we use test_df as input
new_pred = new_pipeline.predict(test_df)

In [ ]:
new_pred.head(5)

In [ ]:
# evaluate test_df
mse, rs = new_pipeline.evaluate(test_df, metric=["mean_squared_error", "r_square"])
print("Evaluate: the mean square error is", mse)
print("Evaluate: the r_square value is", rs)